In [3]:
import os

# Create templates directory
templates_dir = 'templates'
os.makedirs(templates_dir, exist_ok=True)

# HTML content for all templates
templates = {
    'index.html': """
<!-- Model upload form -->
<form action="/upload_model" method="post" enctype="multipart/form-data" class="mb-4">
    <label for="model_file" class="form-label">Upload New Model (.pkl):</label>
    <input type="file" class="form-control" id="model_file" name="model_file" accept=".pkl">
    <button type="submit" class="btn btn-success mt-2">Upload Model</button>
</form>

<!-- Model selection dropdown -->
<form action="/" method="get" class="mb-4">
    <label for="selected_model" class="form-label">Select Model:</label>
    <select class="form-select" id="selected_model" name="selected_model" required>
        {% for model in model_list %}
            <option value="{{ model }}" {% if model == selected_model %}selected{% endif %}>{{ model }}</option>
        {% endfor %}
    </select>
    <button type="submit" class="btn btn-primary mt-2">Use This Model</button>
</form>

{% if model_loaded %}
<div class="alert alert-success">Model loaded and ready for prediction.</div>
{% else %}
<div class="alert alert-warning">No model loaded. Please upload a model to enable prediction.</div>
{% endif %}

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Heart Disease Predictor</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
<div class="container mt-5">
    <h2 class="mb-4">Heart Disease Predictor</h2>
    <form action="/predict" method="post">
        <input type="hidden" name="selected_model" value="{{ selected_model }}">
        <div class="row">
            <div class="mb-3 col-md-4">
                <label for="age" class="form-label">Age (e.g., 45)</label>
                <input type="number" class="form-control" id="age" name="age" required>
            </div>
            <div class="mb-3 col-md-4">
                <label for="cp" class="form-label">Chest Pain Type (0 = Typical Angina, 1 = Atypical Angina, 2 = Non-anginal Pain, 3 = Asymptomatic)</label>
                <input type="number" min="0" max="3" class="form-control" id="cp" name="cp" required>
            </div>
        </div>
        <div class="row">
            <div class="mb-3 col-md-4">
                <label for="trestbps" class="form-label">Resting Blood Pressure (in mm Hg, e.g., 130)</label>
                <input type="number" class="form-control" id="trestbps" name="trestbps" required>
            </div>
            <div class="mb-3 col-md-4">
                <label for="chol" class="form-label">Serum Cholesterol (in mg/dl, e.g., 250)</label>
                <input type="number" class="form-control" id="chol" name="chol" required>
            </div>
        </div>
        <div class="row">
            <div class="mb-3 col-md-4">
                <label for="thalach" class="form-label">Max Heart Rate Achieved (e.g., 160)</label>
                <input type="number" class="form-control" id="thalach" name="thalach" required>
            </div>
            <div class="mb-3 col-md-4">
                <label for="exang" class="form-label">Exercise Induced Angina (1 = Yes, 0 = No)</label>
                <input type="number" min="0" max="1" class="form-control" id="exang" name="exang" required>
            </div>
        </div>
        <div class="row">
            <div class="mb-3 col-md-4">
                <label for="oldpeak" class="form-label">Oldpeak (ST depression induced by exercise, e.g., 1.4)</label>
                <input type="number" step="0.1" class="form-control" id="oldpeak" name="oldpeak" required>
            </div>
            <div class="mb-3 col-md-4">
                <label for="slope" class="form-label">Slope of the Peak Exercise ST Segment (0 = Upsloping, 1 = Flat, 2 = Downsloping)</label>
                <input type="number" min="0" max="2" class="form-control" id="slope" name="slope" required>
            </div>
        </div>
        <button type="submit" class="btn btn-primary">Predict</button>
    </form>
</div>
</body>
</html>
""",
    'result.html': """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Prediction Result</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
    <div class="container mt-5">
        <h2>Prediction Result</h2>
        <div class="alert alert-{{ 'danger' if prediction==1 else 'success' }}">
            {% if prediction == 1 %}
                <strong>Warning:</strong> The model predicts you may have heart disease.
            {% else %}
                <strong>Good News!</strong> The model predicts you are unlikely to have heart disease.
            {% endif %}
        </div>
        <a href="/" class="btn btn-secondary">Try Again</a>
    </div>
</body>
</html>
""",
    'error.html': """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Error</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
    <div class="container mt-5">
        <h2>Error</h2>
        <div class="alert alert-danger">
            {{ error }}
        </div>
        <a href="/" class="btn btn-primary">Back to Home</a>
    </div>
</body>
</html>
"""
}

# Write each template file
for filename, content in templates.items():
    with open(os.path.join(templates_dir, filename), 'w') as f:
        f.write(content)

print("All HTML templates have been created successfully in the 'templates' directory.")


All HTML templates have been created successfully in the 'templates' directory.


In [ ]:
from flask import Flask, request, render_template, redirect, url_for
import joblib
import os
from pyngrok import ngrok
import numpy as np

app = Flask(__name__)
app.config['MODEL_FOLDER'] = 'models'
os.makedirs(app.config['MODEL_FOLDER'], exist_ok=True)

# Ngrok authentication token (use your own token)
NGROK_AUTH_TOKEN = "2xSwaIWShonTP5aqpFLxkgssh5x_2yeVYjMypzRvXD828bQfk"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(5001).public_url
print(f"Ngrok tunnel '{public_url}' -> 'http://localhost:5001'")

def get_model():
    model_path = os.path.join(app.config['MODEL_FOLDER'], 'svc.pkl')
    if os.path.exists(model_path):
        return joblib.load(model_path)
    else:
        return None

@app.route('/', methods=['GET'])
def home():
    # List all .pkl models in the models folder
    model_list = [f for f in os.listdir(app.config['MODEL_FOLDER']) if f.endswith('.pkl')]
    selected_model = request.args.get('selected_model') or (model_list[0] if model_list else None)
    model_loaded = selected_model is not None
    return render_template('index.html', model_loaded=model_loaded, model_list=model_list, selected_model=selected_model)

def get_model(selected_model):
    if selected_model:
        model_path = os.path.join(app.config['MODEL_FOLDER'], selected_model)
        if os.path.exists(model_path):
            return joblib.load(model_path)
    return None

@app.route('/predict', methods=['POST'])
def predict():
    selected_model = request.form.get('selected_model')
    model = get_model(selected_model)
    if model is None:
        return render_template('error.html', error="Model not loaded. Please select or upload a model first.")

    try:
        fields = ['age', 'cp', 'trestbps', 'chol', 'thalach', 'exang', 'oldpeak', 'slope']
        input_data = []
        for field in fields:
            value = request.form.get(field)
            if value is None or value == '':
                raise ValueError(f"Missing value for {field}")
            input_data.append(float(value))
        features = np.array([input_data])

        pred = model.predict(features)[0]
        return render_template('result.html', prediction=int(pred))
    except Exception as e:
        print(f"Prediction error: {e}")
        return render_template('error.html', error=str(e))

@app.route('/upload_model', methods=['POST'])
def upload_model():
    file = request.files.get('model_file')
    if file and file.filename.endswith('.pkl'):
        file.save(os.path.join(app.config['MODEL_FOLDER'], file.filename))
        return redirect(url_for('home'))
    else:
        return render_template('error.html', error="Please upload a valid .pkl file.")

if __name__ == '__main__':
    app.run(port=5001)

Ngrok tunnel 'https://08ab-39-45-22-161.ngrok-free.app' -> 'http://localhost:5001'
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [26/May/2025 20:11:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2025 20:12:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2025 20:12:19] "POST /upload_model HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2025 20:12:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2025 20:12:27] "POST /upload_model HTTP/1.1" 302 -
127.0.0.1 - - [26/May/2025 20:12:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2025 20:12:28] "GET /?selected_model=gradient_boosting.pkl HTTP/1.1" 200 -
c:\Users\Muzam\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
127.0.0.1 - - [26/May/2025 20:13:00] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2025 20:13:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2025 20:13:10] "POST /upload_model HTTP/1.1" 302 -
127.0.0.1 - - [26/May/2025 20:13:10] 